# ETF - Strategy Backtesting (2012-12-03 to 2017-11-30)

### Strategy:
1. At the start of each week, check the 30 week (6 months) rate of change of NiftyBees, GoldBees, BankBees and ONGC.
2. If none of the ETFs are returning more than 3% in 6 months. Do nothing. Be invested in LiquidBees.
3. If one or both ETFs are returning more than 3% invest your entire position in that ETF. (Higher Performance ETF).

In [55]:
import pandas as pd
import quandl

In [56]:
quandl_key = open("data/quandlkey").readline().rstrip()
quandl.ApiConfig.api_key = quandl_key

In [57]:
df = quandl.get(["NSE/NIFTYBEES.5",
                 "NSE/GOLDBEES.5",
                 "NSE/LIQUIDBEES.5",
                 "NSE/BANKBEES.5",
                 "NSE/ONGC.5"]).rename(columns={'NSE/NIFTYBEES - Close': 'nc',
                                                'NSE/GOLDBEES - Close': 'gc',
                                                'NSE/LIQUIDBEES - Close': 'lc',
                                                'NSE/BANKBEES - Close': 'bc',
                                                'NSE/ONGC - Close': 'oc'})

In [58]:
df = df.dropna()
df['wn'] = df.index.weekofyear
print(df)

                 nc       gc       lc       bc      oc  wn
Date                                                      
2007-03-19   369.79   947.80   999.99   499.00  793.15  12
2007-03-21   379.69   945.67   999.99   533.00  811.65  12
2007-03-22   390.03   950.15  1000.00   549.50  852.30  12
2007-03-23   390.19   947.18   999.99   553.00  843.10  12
2007-03-26   386.76   940.71   999.99   547.21  850.25  13
2007-03-29   381.67   946.90   999.99   526.00  874.80  13
2007-04-02   369.51   944.82  1000.00   502.75  826.10  14
2007-04-03   371.67   940.38  1000.01   492.60  843.80  14
2007-04-04   374.37   939.62  1000.00   504.00  847.65  14
2007-04-05   378.36   946.37  1000.00   521.00  846.15  14
2007-04-09   386.61   945.83  1000.00   526.00  871.35  15
2007-04-10   386.29   948.79   999.99   529.00  874.35  15
2007-04-11   387.55   948.89  1000.00   539.00  880.55  15
2007-04-12   386.68   948.18  1000.00   528.00  850.85  15
2007-04-13   392.69   948.41   999.99   543.00  872.35  

### Data Wrangling - Daily to Weekly

Data wrangling performed to compress the large dataset to retain only the essential data points (week beginning and ending).
* week ending - for calculating Rate of Change (RoC)
* week beginning - for trade actions (buy/sell)

In [59]:
# Converting data from df to weekly format

weekly_col = ['wb','we','nb','ne','gb','ge','lb','le', 'bb', 'be', 'ob', 'oe']
weekly = pd.DataFrame(columns=weekly_col)
curr_week = -1
df_prev_week = 0
# cw_data - current week data
cw_data = dict.fromkeys(weekly_col,0.0)

for row in range(len(df)):
    # cw_ds - current week dataset
    cw_ds = df.iloc[row]
    if df_prev_week == cw_ds[5]:
        cw_data['we'] = df.index[row]
        cw_data['ne'] = cw_ds[0]
        cw_data['ge'] = cw_ds[1]
        cw_data['le'] = cw_ds[2]
        cw_data['be'] = cw_ds[3]
        cw_data['oe'] = cw_ds[4]
    else:
        weekly.loc[curr_week] = cw_data
        curr_week += 1
        cw_data['wb'] = cw_data['we'] = df.index[row]
        cw_data['nb'] = cw_data['ne'] = cw_ds[0]
        cw_data['gb'] = cw_data['ge'] = cw_ds[1]
        cw_data['lb'] = cw_data['le'] = cw_ds[2]
        cw_data['bb'] = cw_data['be'] = cw_ds[3]
        cw_data['ob'] = cw_data['oe'] = cw_ds[4]
    df_prev_week = cw_ds[5]

weekly.loc[curr_week] = cw_data
weekly.drop([-1], inplace=True)
print("Weekly Data:")
weekly['wb'] = pd.to_datetime(weekly['wb'])
weekly['we'] = pd.to_datetime(weekly['we'])
print(weekly)

Weekly Data:
            wb         we       nb       ne       gb       ge       lb  \
0   2007-03-19 2007-03-23   369.79   390.19   947.80   947.18   999.99   
1   2007-03-26 2007-03-29   386.76   381.67   940.71   946.90   999.99   
2   2007-04-02 2007-04-05   369.51   378.36   944.82   946.37  1000.00   
3   2007-04-09 2007-04-13   386.61   392.69   945.83   948.41  1000.00   
4   2007-04-17 2007-04-20   402.85   408.07   943.37   946.90   999.99   
5   2007-04-23 2007-04-27   412.59   412.99   946.20   910.08  1000.00   
6   2007-04-30 2007-05-03   411.99   418.99   921.58   914.63  1000.00   
7   2007-05-08 2007-05-09   412.18   410.45   921.26   921.30   999.99   
8   2007-05-14 2007-05-18   416.87   425.09   908.30   888.93  1000.00   
9   2007-05-21 2007-05-25   429.72   426.60   885.22   875.39   999.99   
10  2007-05-28 2007-06-01   431.55   434.61   875.84   883.69  1000.00   
11  2007-06-04 2007-06-08   431.19   419.80   889.31   885.35  1000.00   
12  2007-06-11 2007-06-15

### Trade Simulation

In [60]:
initial_funds = 10000.00
fund_bal = initial_funds
trade_book = pd.DataFrame(columns = ['Date','Scrip','Value','Quantity','Amount'])
# trade book entry
tb_entry = dict.fromkeys(['Date','Scrip','Value','Quantity','Amount'])
period = 30
nbr = gbr = bbr = onr = 0
entry_num = 0

def roc_calc(col):
    initial = weekly.loc[week_num - period][col]
    final = weekly.loc[week_num - 1][col]
    return (final - initial) * 100 / initial

def invest_in(scrip):
    
    def entry_scrip():
        global entry_num, fund_bal
        tb_entry['Date'] = weekly.wb.values[week_num]
        tb_entry['Scrip'] = scrip
        tb_entry['Value'] = weekly[scrip+'b'].values[week_num]
        tb_entry['Quantity'] = int(fund_bal / tb_entry['Value'])
        tb_entry['Amount'] = tb_entry['Value'] * tb_entry['Quantity'] * -1
        fund_bal += tb_entry['Amount']
        # print(tb_entry, fund_bal)
        trade_book.loc[entry_num] = tb_entry
        entry_num += 1
    def exit_scrip():
        global entry_num, fund_bal
        tb_entry['Date'] = weekly.wb.values[week_num]
        tb_entry['Scrip'] = trade_book.Scrip.values[-1]
        tb_entry['Value'] = weekly[trade_book.Scrip.values[-1]+'b'].values[week_num]
        tb_entry['Quantity'] = trade_book.Quantity.values[-1]
        tb_entry['Amount'] = tb_entry['Value'] * tb_entry['Quantity']
        fund_bal += tb_entry['Amount']
        # print(tb_entry, fund_bal)
        trade_book.loc[entry_num] = tb_entry
        entry_num += 1
    
    if entry_num == 0:
        entry_scrip()
    elif scrip != trade_book.Scrip.values[-1]:
        exit_scrip()
        entry_scrip()
        

for week_num in range(period,len(weekly)):
    nbr = roc_calc('ne')
    gbr = roc_calc('ge')
    bbr = roc_calc('be')
    onr = roc_calc('oe')
    
    if nbr > 3:
        if nbr > gbr and nbr > bbr and nbr > onr:
            invest_in('n')
    elif gbr > 3:
        if gbr > bbr and gbr > onr:
            invest_in('g')
    elif bbr > 3:
        if bbr > onr:
            invest_in('b')
    elif onr > 3:
        invest_in('o')
    else:
        invest_in('l')
        
print(trade_book)

         Date Scrip    Value Quantity    Amount
0  2008-03-24     g  1194.98        8  -9559.84
1  2008-08-18     g  1172.38        8   9379.04
2  2008-08-18     o  1074.40        9  -9669.60
3  2008-08-25     o  1014.55        9   9130.95
4  2008-08-25     l   999.99        9  -8999.91
5  2008-09-08     l   999.99        9   8999.91
6  2008-09-08     o  1099.70        8  -8797.60
7  2008-09-15     o   961.35        8   7690.80
8  2008-09-15     l  1000.00        8  -8000.00
9  2008-09-22     l   999.99        8   7999.92
10 2008-09-22     o  1059.70        7  -7417.90
11 2008-10-06     o   981.30        7   6869.10
12 2008-10-06     l  1000.00        7  -7000.00
13 2008-10-13     l  1000.00        7   7000.00
14 2008-10-13     g  1292.67        5  -6463.35
15 2008-10-20     g  1250.67        5   6253.35
16 2008-10-20     l   999.99        7  -6999.93
17 2008-11-24     l  1000.00        7   7000.00
18 2008-11-24     g  1270.18        5  -6350.90
19 2008-12-08     g  1231.18        5   

### Trade Analysis

In [61]:
ltp = weekly[trade_book.Scrip.values[-1]+'e'].values[-1]
namt = ltp * trade_book.Quantity.values[-1]
nbal = namt + fund_bal

odd_dates = pd.Series(trade_book.Date.iloc[1::2].values)
odd_dates.loc[len(odd_dates)] = weekly.we.values[-1]
even_dates = pd.Series(trade_book.Date.iloc[::2].values)
odd_amt = pd.Series(trade_book.Amount.iloc[1::2].values)
odd_amt.loc[len(odd_amt)] = namt
even_amt = pd.Series(trade_book.Amount.iloc[::2].values)

trade_stats = pd.DataFrame()
trade_stats['EntryDate'] = even_dates
trade_stats['ExitDate'] = odd_dates
trade_stats['HoldPeriod'] = odd_dates - even_dates
trade_stats['Scrip'] = pd.Series(trade_book.Scrip.iloc[::2].values)
trade_stats['EntryAmt'] = even_amt
trade_stats['ExitAmt'] = odd_amt
trade_stats['Gains'] = odd_amt + even_amt
print(trade_stats)

    EntryDate   ExitDate HoldPeriod Scrip  EntryAmt   ExitAmt    Gains
0  2008-03-24 2008-08-18   147 days     g  -9559.84   9379.04  -180.80
1  2008-08-18 2008-08-25     7 days     o  -9669.60   9130.95  -538.65
2  2008-08-25 2008-09-08    14 days     l  -8999.91   8999.91     0.00
3  2008-09-08 2008-09-15     7 days     o  -8797.60   7690.80 -1106.80
4  2008-09-15 2008-09-22     7 days     l  -8000.00   7999.92    -0.08
5  2008-09-22 2008-10-06    14 days     o  -7417.90   6869.10  -548.80
6  2008-10-06 2008-10-13     7 days     l  -7000.00   7000.00     0.00
7  2008-10-13 2008-10-20     7 days     g  -6463.35   6253.35  -210.00
8  2008-10-20 2008-11-24    35 days     l  -6999.93   7000.00     0.07
9  2008-11-24 2008-12-08    14 days     g  -6350.90   6155.90  -195.00
10 2008-12-08 2008-12-22    14 days     l  -6999.93   7000.00     0.07
11 2008-12-22 2009-01-19    28 days     g  -6394.45   6559.00   164.55
12 2009-01-19 2009-01-27     8 days     l  -7000.00   7000.00     0.00
13 200

In [62]:
np = trade_stats.Gains.sum()
cagr = (((nbal / initial_funds) ** (1 / 10) -1) * 100).round(2)

print("Initial Funds:\t{0:.2f}".format(initial_funds))
print("Net Profit:\t{0:.2f}".format(np))
print("CAGR:\t\t{0:8.3}%".format(cagr))

Initial Funds:	10000.00
Net Profit:	12736.08
CAGR:		    8.56%
